In [ ]:
import matplotlib.pyplot as plt
%pylab inline
import pandas as pd
import numpy as np
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import datetime
import pprint

In [ ]:
import os.path
import sys
import importlib

pierres_dir = '/net/nfs/srv/MachinePhysics/MachineDevelopment/schnizer/github/'
if pierres_dir not in sys.path:
    sys.path.append(pierres_dir)

import bact2.pandas.dataframe.df_aggregate as dfg
importlib.reload(dfg)

from bact2.applib.utils import steerer as st_u, utils as m_u
from bact2.applib.response_matrix import bpm_data as bd
importlib.reload(st_u)
importlib.reload(m_u)
importlib.reload(bd)

In [ ]:
use_json = True

In [ ]:
if not use_json:
    from databroker import Broker
    db = Broker.named('light')
    uid = '78f148ec-4f55-44c8-bde4-e389ea6ec4d3'
    uid = 'ee52c929-4d97-4838-b007-517beb58cba7'
    uid = '02fe5d1c-b012-4054-8e53-77c911b97e3b'
    uid = '75c454ca-bda5-4254-b7af-0acf365e989d'
    r = db(plan_name='response_matrix')
    cnt = 0
    for doc in r:
        start = doc['start']
        d = datetime.datetime.fromtimestamp(start['time'])
        print(d, start['uid'])

In [ ]:
# First full run 
uid = '5af3a89f-c496-4d3d-84fd-1ca76c9331dc'
# second full run 


In [ ]:
if not use_json:
    header = db[uid]

In [ ]:
path = '/opt/OPI/MachinePhysics/MeasurementData/Users/schnizer/response_matrix'
path = os.getcwd()
path

In [ ]:
pickle_name = os.path.join(path, 'response_measurement.json.bz2')
pickle_name

In [ ]:
if use_json:
    t_table = pd.read_json(pickle_name)
else:
    t_table = header.table()
    t_table.tail()

In [ ]:
t_table.time.max()

In [ ]:
t_table = t_table.infer_objects().sort_index()

In [ ]:
nt = dfg.df_vectors_convert(t_table).sort_index()

In [ ]:
tt = np.array([datetime.datetime.fromtimestamp(t/1e3) for t in t_table.time.values])

In [ ]:
t_table = nt

In [ ]:
def provide_steerer_sort(df):
    tmp = [st_u.SteererSort(*tup) for tup in zip(df.sc_selected, df.index)]
    tmp.sort()
    counter = m_u.Counter()
    for t in tmp:
        t.count = counter(t.name)
    return tmp


Used to get the average position

In [ ]:
cols_to_process = [
    'time',
    'sc_selected',
    'bpm_x_ref', 'bpm_y_ref',
    'bpm_x_diff', 'bpm_y_diff',
    'bk_dev_mode', 
    'steerer_mode',
    'steerer_pos',
    'measurement',
    'I_rounded',
    'bk_dev_dI', 'bk_dev_scale_factor', 'bk_dev_current_offset', 
    'sc_sel_dev_setpoint', 'sc_sel_dev_readback',
    'ramp_index',
    'bpm_waveform_x_pos', 'bpm_waveform_y_pos', 
    'bpm_waveform_ds',
    'dt',
    'bpm_x_scale', 'bpm_y_scale',
    'steerer_type'
]

In [ ]:
def round_float_values(values, rounding_scale=1e6):
    values = np.asarray(values)
    nvals = values * rounding_scale
    nvals = nvals.astype(np.int_)
    nvals = nvals / rounding_scale
    return nvals

In [ ]:
table_sel = t_table.reindex(columns=cols_to_process,index=t_table.index)
table_sel.loc[:,'I_rounded'] = round_float_values(table_sel.sc_sel_dev_setpoint)
table_sel.loc[:, 'measurement'] = -1
table_sel.loc[:, 'steerer_pos'] = -1
table_sel.loc[:, 'ramp_index'] = -1

st_sort = provide_steerer_sort(table_sel)
for st in st_sort:
    table_sel.loc[st.payload, 'steerer_pos'] = int(st.count)
    if st.type == 'h':
        table_sel.loc[st.payload, 'steerer_type'] = 'horizontal'
    elif st.type == 'v':
        table_sel.loc[st.payload, 'steerer_type'] = 'vertical'
    else:
        raise AssertionError('steerer of type {} (name {}) unknwon'.format(st.type, st.name))

table_sel = table_sel.infer_objects()
table_sel.head()

In [ ]:
tmp = (table_sel.sc_sel_dev_setpoint - table_sel.bk_dev_current_offset) / table_sel.bk_dev_scale_factor / 0.07
tmp.min(), tmp.max()

In [ ]:
tt = m_u.add_measurement_counts(table_sel)
tt.loc[:, 'measurement'] = pd.to_numeric(tt.measurement)
table_agg = dfg.df_with_vectors_mean_skip_first(tt.infer_objects())
assert(tt.shape[0]/table_agg.shape[0]==3)

In [ ]:
table_agg.index

In [ ]:
table_meas = table_agg.loc[table_agg.bk_dev_mode == 'store_data'].infer_objects()
table_meas = m_u.add_measurement_index(table_meas)
meas_ref = bd.calc_bpm_gains(table_meas)#.infer_objects()
table_meas_w_ref = bd.calc_bpm_diff(table_meas, meas_ref)

In [ ]:
table_meas_w_ref.head()

In [ ]:
grp = table_meas.groupby(by='sc_selected')
for steerer_name, index in grp.groups.items():
    print(steerer_name)
    sel = table_meas.loc[index,:]
    break

In [ ]:
meas_ref_s = meas_ref.sort_values(by='steerer_pos')
table_meas_w_ref_s = table_meas_w_ref.sort_values(by='steerer_pos')

In [ ]:
gy_all = np.array(meas_ref_s.bpm_gy.tolist(), dtype=np.float_)
gx_all = np.array(meas_ref_s.bpm_gx.tolist(), dtype=np.float_)

In [ ]:
gx_all.shape

In [ ]:
meas_ref.head()

In [ ]:
indices =  np.arange(gx_all.shape[0])
ds = table_agg.bpm_waveform_ds.iat[0]
DS, Indices = np.meshgrid(ds, indices)

In [ ]:
savename = os.path.join(path, 'response_matrix_x.png')
fig = plt.figure(figsize=[16,12])
ax = Axes3D(fig)
ax.plot_surface(Indices, DS, gx_all, rstride=1, cstride=1, cmap=cm.viridis)
ax.set_xlabel('steerer_index')
ax.set_ylabel('bpm position [m]')
ax.set_zlabel('$g_x$ [mm/A]')
fig.savefig(savename, dpi=100)

In [ ]:
savename = os.path.join(path, 'response_matrix_y.png')
fig = plt.figure(figsize=[16,12])
ax = Axes3D(fig)
ax.plot_surface(Indices, DS, gy_all, rstride=1, cstride=1, cmap=cm.viridis)
ax.set_xlabel('steerer_index')
ax.set_ylabel('bpm position [m]')
ax.set_zlabel('$g_y$ [mm/A]')
fig.savefig(savename, dpi=100)

In [ ]:
def add_steerer_scale(df, column_name='sc_selected', copy=True):
    if copy:
        df = df.copy()

    df.bpm_x_diff = df.bpm_waveform_x_pos - df.bpm_x_ref
    df.bpm_y_diff = df.bpm_waveform_y_pos - df.bpm_y_ref
    
    grp = df.groupby(by='sc_selected')
    for steerer_name, index in grp.groups.items():
        sel = df.loc[index,:]
        sel_ri = sel.loc[sel.ramp_index == 4,:]
        assert(sel_ri.shape[0] == 1)
        
        # scale for the x factor
        s_x = sel_ri.bpm_waveform_x_pos.iat[0]
        # scale for the y factor
        s_y = sel_ri.bpm_waveform_y_pos.iat[0]
        
        li = len(index)
        ss_x = pd.Series(index=index, data=[s_x] * li, dtype=np.object_)
        ss_y = pd.Series(index=index, data=[s_y] * li, dtype=np.object_)
            
        df.loc[index, 'bpm_x_scale'] = ss_x
        df.loc[index, 'bpm_y_scale'] = ss_y
        
    return df
        #break  
    

In [ ]:
table_meas_w_ref = add_steerer_scale(table_meas_w_ref)

In [ ]:
table_meas_w_ref.loc[:, ['time', 'sc_selected', 'ramp_index', 'bpm_x_diff', 'bpm_y_diff', ]]

In [ ]:
steerer_sel = 'hs1pd1r'
df_sel = table_meas_w_ref.loc[table_meas_w_ref.sc_selected==steerer_sel,:]

dx_one = np.array(df_sel.bpm_x_diff.tolist(), dtype=np.float_)
dy_one = np.array(df_sel.bpm_y_diff.tolist(), dtype=np.float_)
dx_s   = np.array(df_sel.bpm_x_scale.tolist(), dtype=np.float_)
dy_s   = np.array(df_sel.bpm_y_scale.tolist(), dtype=np.float_)

In [ ]:
dx_s_one = dx_one/dx_s

In [ ]:
dy_s_one = dy_one/dy_s

In [ ]:
indices =  df_sel.ramp_index
ds = df_sel.bpm_waveform_ds.iat[0]
DS, Indices = np.meshgrid(ds, indices)

In [ ]:
savename = os.path.join(path, 'steerer_inhomogeneity_{}_x.png'.format(steerer_sel))
fig = plt.figure(figsize=[16,12])
ax = Axes3D(fig)
ax.plot_surface(Indices, DS, dx_s_one, rstride=1, cstride=1, cmap=cm.viridis)
ax.set_xlabel('ramp_index')
ax.set_ylabel('bpm position [m]')
ax.set_zlabel('$d_x$ [$\mu$m]')
fig.savefig(savename, dpi=100)

In [ ]:
savename = os.path.join(path, 'steerer_inhomogeneity_{}_y.png'.format(steerer_sel))
fig = plt.figure(figsize=[16,12])
ax = Axes3D(fig)
ax.plot_surface(Indices, DS, dy_s_one * 1000, rstride=1, cstride=1, cmap=cm.viridis)
ax.set_xlabel('ramp_index')
ax.set_ylabel('bpm position [m]')
ax.set_zlabel('$d_y$ [$\mu$m]')
fig.savefig(savename, dpi=100)

In [ ]:
df_sel = table_meas_w_ref.loc[table_meas_w_ref.steerer_type == 'horizontal']
dx_all = np.array(df_sel.bpm_x_diff.tolist(), dtype=np.float_)
dx_s_all = np.array(df_sel.bpm_x_scale.tolist(), dtype=np.float_)
sdx_all = dx_all/dx_s_all

val = 10
sdx_all[np.absolute(sdx_all>val)] = 0


In [ ]:
indices =  df_sel.measurement
ds = df_sel.bpm_waveform_ds.iat[0]
DS, Indices = np.meshgrid(ds, indices)

In [ ]:
savename = os.path.join(path, 'steerer_inhomogeneity_all_x.png')
fig = plt.figure(figsize=[16,12])
ax = Axes3D(fig)
ax.plot_surface(Indices, DS, sdx_all, rstride=1, cstride=1, cmap=cm.viridis)
ax.set_xlabel('ramp_index')
ax.set_ylabel('bpm position [m]')
ax.set_zlabel('$d_x$ [$\mu$m]')
fig.savefig(savename, dpi=100)

In [ ]:
df_sel = table_meas_w_ref.loc[table_meas_w_ref.steerer_type == 'vertical']
dy_all = np.array(df_sel.bpm_y_diff.tolist(), dtype=np.float_)
dy_s_all = np.array(df_sel.bpm_y_scale.tolist(), dtype=np.float_)
sdy_all = dy_all/dy_s_all

val = 1
sdy_all[np.absolute(sdy_all)>val] = 0

In [ ]:
indices =  df_sel.measurement
ds = df_sel.bpm_waveform_ds.iat[0]
DS, Indices = np.meshgrid(ds, indices)

In [ ]:
savename = os.path.join(path, 'steerer_inhomogeneity_all_y.png')
fig = plt.figure(figsize=[16,12])
ax = Axes3D(fig)
ax.plot_surface(Indices, DS, sdy_all, rstride=1, cstride=1, cmap=cm.viridis)
ax.set_xlabel('ramp_index')
ax.set_ylabel('bpm position [m]')
ax.set_zlabel('$sd_x$')
fig.savefig(savename, dpi=100)

In [ ]:
ax.axis()